In [1]:
pip install git+https://github.com/HOUPU1993/tidyzoning.git

  Cloning https://github.com/HOUPU1993/tidyzoning.git to c:\users\admin\appdata\local\temp\pip-req-build-tfacdejx
  Resolved https://github.com/HOUPU1993/tidyzoning.git to commit 83c89410163f9246b747db2899be3999c01cd08a
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for tidyzoning: filename=tidyzoning-0.1.0-py3-none-any.whl size=2470 sha256=acb5fb8af0acb1967bad550b94da9369df1556997987647f2925560a9b6a3ac6
  Stored in directory: C:\Users\Admin\AppData\Local\Temp\pip-ephem-wheel-cache-z2b8zu4d\wheels\f4\29\51\e431eb95e0bf6589310375ce483daada9c916a6f65f3b65a8f
Successfully built tidyzoning
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/HOUPU1993/tidyzoning.git 'C:\Users\Admin\AppData\Local\Temp\pip-req-build-tfacdejx'


In [1]:
import tidyzoning

TypeError: 'module' object is not callable

In [ ]:
hello_world()

NameError: name 'hello_world' is not defined